In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from datetime import datetime, timedelta

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
#first_row = session.query(Measurement.date).first()
most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
most_recent_date

In [ ]:
# Calculate the date one year from the last date in data set.
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query Date: ", query_date)


In [ ]:
# Calculate the date 12 months ago from the current date
#current_date = pd.Timestamp.now()
#date_12_months_ago = current_date - pd.DateOffset(months=12)

# Query the database for precipitation data from the last 12 months
end_date = datetime.now()
start_date = end_date - timedelta(365)

#results = session.query(Measurement, Measurement.prcp).filter(
    #Measurement.date >= start_date,
    #Measurement.date <= end_date
#).all()
results = session.query(Measurement).filter(Measurement.date.between(start_date, end_date)).all()

for row in results:
    print(row)

query = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > '2016-08-23').\
    order_by(Measurement.date.desc() ).all()

In [ ]:
df = pd.DataFrame(query, columns=['Date', 'Precipitation'])
df

In [ ]:
df.sort_values(by='Date', inplace=True)
df

In [ ]:
dk = pd.DataFrame(df)

# Remove NaN and empty cells
df_cleaned = dk.dropna().replace('', pd.NA).dropna()

print(df_cleaned)

In [ ]:
# Plotting bar chart
df_cleaned.plot(x='Date', y='Precipitation', kind='bar', color='skyblue')

# Adding labels and title
plt.xlabel('Date')
plt.ylabel('Precipitation')
plt.title('Precipitation by Date')

# Rotate x-axis labels for better readability
plt.xticks(rotation=90)

# Display the bar chart
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
summary_stats = df['Precipitation'].describe()
print(summary_stats)

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
total_stations = session.query(Station).count()

print("Total number of stations:", total_stations)

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
most_active_stations = session.query(Measurement.station, func.count(Measurement.station).label('activity_count')) \
                                .group_by(Measurement.station) \
                                .order_by(func.count(Measurement.station).desc()) \
                                .all()
most_active_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
#from sqlalchemy import func
#from your_module import Temperature, session

# Assuming you have a SQLAlchemy session object named `session` and a Temperature model defined

# Get the most active station ID from the previous query
most_active_station_id = 'USC00519281'

# Query to calculate lowest, highest, and average temperature for the most active station
temperature_stats = session.query(func.min(Measurement.tobs).label('lowest_temp'),
                                  func.max(Measurement.tobs).label('highest_temp'),
                                  func.avg(Measurement.tobs).label('average_temp')) \
                           .filter(Measurement.station == most_active_station_id) \
                           .first()

# Printing the results
lowest_temp, highest_temp, average_temp = temperature_stats
print(f"Lowest Temperature: {lowest_temp}")
print(f"Highest Temperature: {highest_temp}")
print(f"Average Temperature: {average_temp}")


In [ ]:
# import matplotlib.pyplot as plt
#from sqlalchemy import func, and_
#from datetime import datetime, timedelta
#from your_module import Temperature, session

# Assuming you have a SQLAlchemy session object named `session` and a Temperature model defined

# Get the most active station ID from the previous query
most_active_station_id =  'USC00519281'

# Calculate the date 12 months ago
one_year_ago = datetime.now() - timedelta(days=365)

# Query temperature observation data for the most active station within the last 12 months
temperature_data = session.query(Measurement.tobs) \
                          .filter(Measurement.station == most_active_station_id,
                                       Measurement.date >= '2016-08-23') \
                          .all()
temperature_data
# Extract temperature values from the query result
#temperatures = [temp[0] for temp in temperature_data]
 
# Plot the results as a histogram
plt.hist(temperature_data, bins=20, color='skyblue',  edgecolor='black')
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.title('Temperature Distribution for the Last 12 Months')
plt.grid(True)
plt.show()


# Close Session

In [ ]:
# Close Session
session.close()